### CitiBike ML Demo
By: David Stearns  
  
This demo (while fully functional *could use a bit of clean up!* :D ) demonstrates Snowpark Python's capability to build, train, and deploy machine learning models. It uses the Scalar UDF function which is great for single predictions, however, if you would like to do full table predictions or use input batches, the best way to accomplish that is to use Vectorized UDF's. 

In [18]:
import snowflake.snowpark
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.functions import udf
from snowflake.snowpark.session import Session
from snowflake.snowpark import types as T
import json
import pandas as pd
import numpy as np
import datetime
import sys
import math
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark import Window
pd.set_option('display.max_columns', None)
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
import credentials
import io
import joblib
import cachetools
#
from multiprocessing import pool
#
conn = {
    "account": credentials.credentials["account"],
    "user": credentials.credentials["username"],
    "password": credentials.credentials["password"],
    "role": credentials.credentials["role"],
    "warehouse": credentials.credentials["warehouse"],
    "database": credentials.credentials["database"],
    "schema": credentials.credentials["schema"]
}
session = Session.builder.configs(conn).create()

In [51]:
def save_model(session, model, path):
  input_stream = io.BytesIO()
  joblib.dump(model, input_stream)
  session._conn._cursor.upload_stream(input_stream, path)
  return "successfully created file: " + path

In [52]:
session.sql(f"create or replace stage citibike_ml").collect()

[Row(status='Stage area CITIBIKE_ML successfully created.')]

In [53]:
df_trips_hourly = session.table("trips_stations_vw")
print("Aggregating TRIPS data")
print("-------------------------------------------")
df_trips_daily = df_trips_hourly.select(F.to_date(F.col("STARTTIME")).as_("ds")).groupBy(F.col("ds")).count().select(F.col("ds"), F.col("COUNT").as_("y"))
df_weather = session.table("weather_vw")
print("Joining the aggregated TRIPS data with WEATHER data from the Snowflake Data Marketplace")
print("-------------------------------------------")
df_trips_weather = df_trips_daily.join(df_weather, df_trips_daily["DS"] == df_weather["OBSERVATION_DATE"])\
                    .select(F.col("DS"),F.col("Y"), F.col("TEMP_AVG_C"), F.col("TOT_PRECIP_IN"))
print("Feature engineering: creating an indicator column for rain (1 = rain, 0 = no rain)")
print("-------------------------------------------")
df_trips_weather = df_trips_weather.withColumn("rain_indicator", F.when(F.col("TOT_PRECIP_IN") > 0, F.lit(1) ).otherwise(F.lit(0)))
print("Sorting the table by date")
print("-------------------------------------------")
df_trips_weather = df_trips_weather.sort(F.col("DS"))
print("Writing back the table to the Snowflake database with selected columns")
print("-------------------------------------------")
df_trips_weather.select("DS", "Y", "RAIN_INDICATOR", "TEMP_AVG_C").write.mode("overwrite").save_as_table("citibike_ml_demo")

In [ ]:
cb_ml = session.table("citibike_ml_demo")

In [ ]:
print("Feature engineering: creating a one day shift and natural log transformation","\n",
        "of the TRIPS figure (target) and saving it back to the Snowflake database","\n",
        "to create a features table")
print("-------------------------------------------")
session.sql(""" select
                    RAIN_INDICATOR,
                    TEMP_AVG_C,
                    Y_log,
                    LAG(Y_log,1,0) OVER (order by DS) as Y_yesterday
                from (
                    select ds, rain_indicator, temp_avg_c, round(ln(Y), 2) as Y_log from citibike_ml_demo
                ) A""").write.mode("overwrite").save_as_table("citibike_ml_demo_features")

In [ ]:
session.table("citibike_ml_demo_features").show()

This next line of code shows feature engineering happening inside of this Stored Procedure in a Pandas dataframe. I would actually recommend against this (I just did this in a time crunch). The best way to do this is to create a stored procedure using a standard warehouse to run all of your feature transformations NOT using Pandas (so, SQL or just using the Snowpark Dataframe API syntax), and save that table. Once you have your final table, you can then create a Stored Procedure using a High Memory Warehouse to run the model training flow.

In [16]:
cb_ml = session.table("citibike_ml_demo")
cb_ml.show(5)

--------------------------------------------------------------
|"DS"        |"Y"    |"RAIN_INDICATOR"  |"TEMP_AVG_C"        |
--------------------------------------------------------------
|2020-09-24  |21155  |0                 |20.064              |
|2020-09-25  |20186  |0                 |20.116              |
|2020-09-26  |20306  |1                 |20.488              |
|2020-09-27  |18125  |1                 |22.362000000000002  |
|2020-09-28  |16108  |1                 |22.259999999999998  |
--------------------------------------------------------------



In [56]:
session.add_packages('snowflake-snowpark-python', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'cachetools')
def citibike_ml_demo(session: snowflake.snowpark.Session) -> str:

    import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F
    from snowflake.snowpark import types as T

    model_name = "predict_trips"
    stage_name = "citibike_ml"
    df = session.table("citibike_ml_demo_features")
    pandas_df = df.toPandas()
    pandas_df = pandas_df[1:]

    X = pandas_df[["RAIN_INDICATOR", "TEMP_AVG_C", "Y_YESTERDAY"]]
    y = pandas_df[["Y_LOG"]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33 )
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X_train, y_train)

    save_model(session, rf, f"@{stage_name}/{model_name}.joblib")

    model_name_full = session.sql(f"ls @{stage_name}").collect()[0][0]

    performance = cross_validate(rf, X, y, cv=3)

    return f"Created model: {model_name_full}.... Performance Metrics: {performance}"

In [57]:
# Registering my SPROC within Snowflake
train_rfc_model = session.sproc.register(citibike_ml_demo, replace=True)

In [58]:
# Running the SPROC for model training
train_rfc_model(session=session)

"Created model: citibike_ml/predict_trips.joblib.... Performance Metrics: {'fit_time': array([0.13583279, 0.13426924, 0.13235784]), 'score_time': array([0.01061487, 0.00998688, 0.01009941]), 'test_score': array([0.95149652, 0.9094952 , 0.93581312])}"

In [59]:
# Importing my model into the session
session.add_import("@citibike_ml/predict_trips.joblib")  

# Caching my model so it is only read once
@cachetools.cached(cache={})

# Function to read the model file into another function
def read_file(filename):
       import os
       import_dir = sys._xoptions.get("snowflake_import_directory")
       if import_dir:
              with open(os.path.join(import_dir, filename), 'rb') as file:
                     m = joblib.load(file)
                     return m

In [60]:
# Registering the UDF
@udf(session=session, name="predict_trips", is_permanent=True, stage_location="@citibike_ml", replace=True)
def predict_trips(df: T.PandasDataFrame[int, float, float]) -> T.PandasSeries[float]:
    m = read_file('predict_trips.joblib')
    if isinstance(m, str):
        return m
    return m.predict(df)

In [67]:
test_df = session.sql("select rain_indicator, temp_avg_c, y_yesterday from citibike_ml_demo_features")

In [69]:
results = test_df.select(
    *test_df,
    F.ceil(F.call_udf("predict_trips", *test_df)).alias('PREDICTION')
    ).where(F.col("Y_YESTERDAY") != 0).select(
        *test_df,
        F.col("PREDICTION"),
        F.when(
            F.col("PREDICTION") != F.col("Y_YESTERDAY"),
            ((F.col("PREDICTION")-F.col("Y_YESTERDAY"))/F.col("Y_YESTERDAY"))*100
        ).alias("ERROR_PERCENT")
            )

In [70]:
results.show()

------------------------------------------------------------------
|"RAIN_INDICATOR"  |"TEMP_AVG_C"  |"Y_YESTERDAY"  |"PREDICTION"  |
------------------------------------------------------------------
|1                 |27.52         |0.0            |9.0           |
|1                 |24.996        |9.8            |10.0          |
|0                 |26.174        |9.83           |10.0          |
|0                 |24.226        |9.81           |10.0          |
|1                 |20.406        |9.78           |10.0          |
|1                 |21.924        |9.69           |10.0          |
|1                 |23.54         |9.62           |10.0          |
|1                 |21.45         |9.75           |10.0          |
|0                 |22.274        |9.82           |10.0          |
|0                 |24.94         |9.86           |10.0          |
------------------------------------------------------------------



In [11]:
# create a prediction
# inputs are: RAIN(1/0), TEMP(in C), TRIPS YESTERDAY(natural log)
# the trips_yesterday variable has to be in the form of a natural log
# you can change these variables if you want... but it doesn't matter

trips_yesterday_log = 10.1
rain = 0
temp = 22

trips_yesterday = math.floor(np.exp(10.1))
print(f"There were {trips_yesterday} trips yesterday")
trips_predicted_today = math.floor(session.sql(f"select exp(predict_trips([{rain}, {temp}, {trips_yesterday_log}]))").collect()[0][0])
print(f"Based on the inputs: \n RAIN = {rain}, TEMP(C) = {temp}, TRIPS YESTERDAY = {trips_yesterday}\n....\nThere will be {trips_predicted_today} trips today.")

There were 24343 trips yesterday
Based on the inputs: 
 RAIN = 0, TEMP(C) = 22, TRIPS YESTERDAY = 24343
....
There will be 22026 trips today.
